In [ ]:
! pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu100/torch_nightly.html

Looking in links: https://download.pytorch.org/whl/nightly/cu100/torch_nightly.html


In [1]:
import torch.onnx

In [2]:
! pip install transformers==3.0.2

     |████████████████████████████████| 778kB 5.8MB/s 
     |████████████████████████████████| 890kB 4.1MB/s 
     |████████████████████████████████| 3.0MB 27.3MB/s 
     |████████████████████████████████| 1.1MB 44.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a31488f54ed23ce8741facb12e65303ff40732e5892e26825052747b44652e79
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [4]:
class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.5)
        self.classifier = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [15]:
model  = DistilBERTClass()

In [16]:
model.load_state_dict(torch.load('/content/drive/MyDrive/distilbert_demo_emotions_state_dict_11_17_epoch2', map_location=torch.device('cpu')), strict=False)

<All keys matched successfully>

In [17]:
model = torch.nn.DataParallel(model)

In [18]:
model.eval()

DataParallel(
  (module): DistilBERTClass(
    (l1): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
              (dropout):

In [19]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [20]:
text = 'this is an challenging task'
input = tokenizer.encode_plus(
              text,
              None,
              add_special_tokens=True,
              max_length=100,
              pad_to_max_length=True,
              return_token_type_ids=True
          )
ids = torch.tensor([input['input_ids']], dtype=torch.long)
mask = torch.tensor([input['attention_mask']], dtype=torch.long)
token_type_ids = torch.tensor([input["token_type_ids"]], dtype=torch.long)


In [22]:
torch.onnx.export(
    model.module,
    (ids, mask, token_type_ids),
    '/content/drive/MyDrive/distilbert_emotions_detection.onnx',
    input_names=['ids', 'mask', 'ttids'],
    output_names= ['output'],
    dynamic_axes={
        'ids': {0: 'batch_size'}, 
        'mask': {0: 'batch_size'}, 
        'ttids': {0: 'batch_size'}, 
        'output': {0: 'batch_size'}
    }
)